In [1]:
import pandas as pd
from xtquant import xtdata
import json
import copy

xtquant文档地址：http://dict.thinktrader.net/nativeApi/start_now.html


In [2]:
class CallBackBase():
    def __init__(self, stock_code, period, start_time, end_time, incrementally=True):
        self.stock_code = stock_code
        self.period = period
        self.start_time = start_time
        self.end_time = end_time
        self.incrementally = incrementally

    def downAndLoadData(self):
        # 下载数据和载入数据
        xtdata.download_history_data(stock_code=self.stock_code, period=self.period, start_time=self.start_time, end_time=self.end_time, incrementally=self.incrementally)
        data = xtdata.get_market_data_ex(stock_list=[self.stock_code], period=self.period, start_time=self.start_time,end_time=self.end_time)
        return data

    def build_iter(self,data):
        pass

    def call(self,start_price, price_interval, stock_num_all, stock_num_wai, stock_num_lei, oneHand):
        return self.check(start_price, price_interval, stock_num_all, stock_num_wai, stock_num_lei, oneHand)

    def check(self,start_price, price_interval, stock_num_all, stock_num_wai, stock_num_lei, oneHand):
        # 重写
        pass

In [3]:
class CallBack_WangGe(CallBackBase):
    def __init__(self,  
                 stock_code, period, start_time, end_time):
        super().__init__(
            stock_code, period, start_time, end_time)

        # 数据载入和整理数据
        self.data = self.downAndLoadData()
        self.data_iter = self.build_iter(self.data)

    def build_iter(self,data):
        # 创建生成器对象
        data = data[self.stock_code].to_dict(orient="records")
        data = [i for i in data if 25200000 > i["time"]%86400000 > 5400000]
        data.sort(key=lambda x: x["time"])
        print(data[0])
        print(data[-1])
        return data

    def guandan(self):
        # 买
        if self.stock_num_wai >= self.oneHand:
            self.buy_price = self.start_price - self.price_interval
        else:
            self.buy_price = None
        # 卖
        if self.stock_num_lei >= self.oneHand:
            self.sale_price = self.start_price + self.price_interval
        else:
            self.sale_price = None

        print("挂单：基准价:{},买:{},卖:{}".format(str(self.start_price),str(self.buy_price),str(self.sale_price)))
    
    def init_conf(self,start_price, price_interval, stock_num_all, stock_num_wai, stock_num_lei, oneHand):
        self.start_price = start_price
        self.price_interval = price_interval
        self.stock_num_all = stock_num_all
        self.stock_num_wai = stock_num_wai
        self.stock_num_lei = stock_num_lei
        self.oneHand = oneHand
        self.shouxufei = 0.001 # 手续费：只在卖的时候收取

        # self.start_price = 7.411 # 起始价格
        # self.price_interval = 0.025 # 价格区间
        # self.stock_num_all = 30000 # 总量
        # self.stock_num_wai = 15000 # 外盘，钱
        # self.stock_num_lei = 15000 # 内盘，票
        # self.oneHand = 5000 # 一手交易数量
        
        self.shouyi = 0.0

    def check(self,start_price, price_interval, stock_num_all, stock_num_wai, stock_num_lei, oneHand):
        # 初始化参数
        self.init_conf(start_price, price_interval, stock_num_all, stock_num_wai, stock_num_lei, oneHand)
        # 挂单
        self.guandan()

        # 开始执行
        for oneline in self.data_iter:
            # 触发买单
            if self.buy_price != None and self.buy_price > float(oneline["lastPrice"]):
                self.stock_num_wai = self.stock_num_wai - self.oneHand
                self.stock_num_lei = self.stock_num_lei + self.oneHand
                self.start_price = self.buy_price
                self.guandan()
            
            # 触发卖单+结算收益
            elif self.sale_price != None and self.sale_price < float(oneline["lastPrice"]):
                #
                self.stock_num_wai = self.stock_num_wai + self.oneHand
                self.stock_num_lei = self.stock_num_lei - self.oneHand
                self.start_price = self.sale_price
                self.guandan()

                #
                self.shouyi = self.shouyi + (self.price_interval-self.shouxufei)*self.oneHand

            else:
                pass
        
        return self.shouyi

In [4]:
class CallBack_1d(CallBackBase):
    def __init__(self,
                 stock_code, period, start_time, end_time):
        super().__init__(
            stock_code, period, start_time, end_time)
        
        # 数据载入和整理数据
        self.data = self.downAndLoadData()
        self.data_iter = self.build_iter(self.data)
    
    def build_iter(self,data):
        data = data[self.stock_code].to_dict(orient='records')
        data.sort(key=lambda x: x["time"])
        return data
    
    def init_conf(self):
        self.summ = 0.0
        self.sample_num = 0
        self.snmm_volume = 0

    def call(self):
        self.init_conf()
        ave,ave_volume = self.check()
        return ave,ave_volume

    def check(self):
        for i in self.data_iter:
            diff = i["high"] - i["low"]
            volume = i["volume"]
            self.summ += diff
            self.snmm_volume += volume
            self.sample_num += 1
        ave = self.summ/self.sample_num
        ave_volume = self.snmm_volume/self.sample_num
        return ave,ave_volume
    

In [3]:
class CallBack_1d_simi(CallBackBase):
    def __init__(self,
                 stock_code, period, start_time, end_time, incrementally):
        super().__init__(
            stock_code, period, start_time, end_time, incrementally)
        
        # 数据载入和整理数据
        self.data = self.downAndLoadData()
        self.data_iter = self.build_iter(self.data)
    
    def build_iter(self,data):
        data = data[self.stock_code].to_dict(orient='records')
        data.sort(key=lambda x: x["time"])
        return data
    
    def init_conf(self,static_num,aim_num,loss_num,close=None):
        self.static_num  = static_num # 样本长度
        self.aim_num = aim_num # 预测数量
        self.loss_num = loss_num # 去损失top_n
        self.test_num = 20
        if close:
            self.data_iter[-1]["close"] = close

    def call(self,mode="callback"):
        pre = self.check(mode=mode)
        return pre

    def msg_loss(self,base,sample,keys):
        loss = 0
        for b,s in zip(base,sample):
            for key in keys:
                    loss = loss + abs(b[key]-s[key])
        return loss

    def divideTestSet(self,sample,mode):
        if mode == "callback":
            test = sample[-self.test_num:]
            sample = sample[:-(self.test_num+self.static_num)]
            
        elif mode == "inference":
            test = [sample[-1][1:] + [sample[-1][0]]]
            sample = sample[:-self.static_num]
            from datetime import datetime
            timestamp = test[0][-2]["time"]/1000
            print("最新时间：",datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S'))
            print(test[0][-2])
            print(self.data_iter[-1])

        return sample,test

        
    def check(self,mode):
        if mode not in ["callback","inference"]:
            print("mode不对，重新输入")

        # 裁剪分类+归一化操作
        sample = []
        for i in range(len(self.data_iter)-(self.static_num + self.aim_num)+1):
            sample.append(self.data_iter[i:i+self.static_num+self.aim_num])

        sample_tmp = []
        for one_com in sample:
            keys = {"open":0.0, "high":0.0, "low":0.0, "close":0.0 }  # volume
            for key in keys:
                for one in one_com[:-self.aim_num]:
                    keys[key] += one[key]
                keys[key] = keys[key] / self.static_num
            one_com_tmp = copy.deepcopy(one_com)
            for key in keys:
                for one in one_com_tmp:
                    one[key] = one[key] - keys[key]
            sample_tmp.append(one_com_tmp)
        self.sample = sample_tmp
        
        # 裁剪测试集和待匹配集和
        self.sample,self.test = self.divideTestSet(self.sample,mode)

        # 统计准确率
        pre = 0


        # 开始匹配
        for one_test in self.test:
            # 目标
            if one_test[-1]["close"] > one_test[-2]["close"]:
                aim = "大于"
            else:
                aim = "小于"

            # 开始匹配
            one_test = one_test
            sample = self.sample
            loss = []
            keys = {"open":0.0, "high":0.0, "low":0.0, "close":0.0}
            for index in range(len(self.sample)):
                tmp = self.msg_loss(one_test[:-self.aim_num], sample[index][:-self.aim_num], keys)
                loss.append([index,tmp])
            loss.sort(key=lambda x: x[1],reverse=False)
            loss = loss[:self.loss_num]

            # 大于收盘价 或 小于收盘价
            dayu = 0
            xiaoyu = 0
            for i in loss:
                if sample[i[0]][-1]["close"] > sample[i[0]][-2]["close"]:
                    dayu += 1
                else:
                    xiaoyu += 1
            if dayu >= xiaoyu:
                predict = "大于{}".format(str(dayu))
            else:
                predict = "小于{}".format(str(xiaoyu))

            if aim in predict:
                pre += 1

        if mode == "inference":
            print("预测结果为：{} 收盘价".format(predict))
            return predict
        else:
            return pre/self.test_num
    
# 寻参数方案：
# 最新20个数据点作为测试集合
# 参数范围：
    # 筛选范围(20250215,20240825,20240215,20230825,20230215,20220825,20220215,....20140825,20140215)
    # 匹配时间长度（1,2,3,4,5）
    # 相似样本取Top_n (2,3,4,5,6,7,8,9,10)
# 计算准确率：

# callback模式
# data_list = ["20250215","20240825","20240215","20230825","20230215","20220825","20220215","20210825","20210215","20200825","20200215",
#                 "20190825","20190215","20180825","20180215","20170825","20170215","20160825","20160215","20150825","20150215","20140825","20140215"]
# static_num_list = [1,2,3,4,5,6,7,8,9,10,11,12]
# loss_num_list = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]

# with open("c:\\Users\\Administrator\\Desktop\\Gold\\result.json","w") as f:
#     for data in data_list:
#         cd = CallBack_1d_simi(
#                 "518880.SH","1d",data,"",True
#             )
#         for static_num in static_num_list:
#             for loss_num in loss_num_list:
#                 cd.init_conf(static_num=static_num,aim_num=1,loss_num=loss_num)
#                 pre = cd.call(mode="callback")
#                 f.write(json.dumps({"data":data,"static_num":static_num,'loss_num':loss_num,"pre":pre})+"\n")

# 前传模式
conf_list = [
 {'data': '20200215', 'static_num': 7, 'loss_num': 5, 'pre': 0.85},
 {'data': '20240825', 'static_num': 7, 'loss_num': 10, 'pre': 0.8},
 {'data': '20240215', 'static_num': 7, 'loss_num': 9, 'pre': 0.8},
 {'data': '20230215', 'static_num': 6, 'loss_num': 19, 'pre': 0.8},
 {'data': '20220215', 'static_num': 4, 'loss_num': 18, 'pre': 0.8},
 {'data': '20220215', 'static_num': 7, 'loss_num': 4, 'pre': 0.8},
 {'data': '20210215', 'static_num': 5, 'loss_num': 14, 'pre': 0.8},
 {'data': '20200825', 'static_num': 5, 'loss_num': 14, 'pre': 0.8},
 {'data': '20200825', 'static_num': 7, 'loss_num': 5, 'pre': 0.8},
 {'data': '20200215', 'static_num': 5, 'loss_num': 16, 'pre': 0.8},
 {'data': '20200215', 'static_num': 7, 'loss_num': 13, 'pre': 0.8},
 {'data': '20190825', 'static_num': 7, 'loss_num': 7, 'pre': 0.8},
 {'data': '20170215', 'static_num': 3, 'loss_num': 11, 'pre': 0.8},
 {'data': '20160825', 'static_num': 3, 'loss_num': 11, 'pre': 0.8},
 {'data': '20160825', 'static_num': 5, 'loss_num': 20, 'pre': 0.8},
 {'data': '20160825', 'static_num': 6, 'loss_num': 16, 'pre': 0.8},
 {'data': '20140215', 'static_num': 5, 'loss_num': 14, 'pre': 0.8}
]

result_list = {
    "大于":0,
    "小于":0,
    "总票数":0
}
for conf in conf_list:
    cd = CallBack_1d_simi(
                "518880.SH","1d",conf["data"],"",True
            )
    cd.init_conf(static_num=conf["static_num"],aim_num=1,loss_num=conf["loss_num"])
    predict = cd.call(mode="inference")
    if "大于" in  predict:
        result_list["大于"] += 1
        result_list["总票数"] += 1
    else:
        result_list["小于"] += 1
        result_list["总票数"] += 1
print(result_list)

***** xtdata连接成功 2025-08-27 21:10:45*****
服务信息: {'tag': 'sp3', 'version': '1.0'}
服务地址: 8.156.70.64:58610
数据路径: C:\Program Files\国金证券QMT交易端\bin.x64/../userdata_mini/datadir
设置xtdata.enable_hello = False可隐藏此消息

最新时间： 2025-08-27 00:00:00
{'time': 1756224000000, 'open': 0.06100000000000083, 'high': 0.047428571428571153, 'low': 0.052142857142857046, 'close': 0.04628571428571515, 'volume': 2359129, 'amount': 1758733831.0, 'settelementPrice': 0.0, 'openInterest': 15, 'preClose': 7.449, 'suspendFlag': 0}
{'time': 1756224000000, 'open': 7.461, 'high': 7.461, 'low': 7.444, 'close': 7.45, 'volume': 2359129, 'amount': 1758733831.0, 'settelementPrice': 0.0, 'openInterest': 15, 'preClose': 7.449, 'suspendFlag': 0}
预测结果为：小于3 收盘价
最新时间： 2025-08-27 00:00:00
{'time': 1756224000000, 'open': 0.06100000000000083, 'high': 0.047428571428571153, 'low': 0.052142857142857046, 'close': 0.04628571428571515, 'volume': 2359129, 'amount': 1758733831.0, 'settelementPrice': 0.0, 'openInterest': 15, 'preClose': 7.449, '

In [4]:
import json
with open("c:\\Users\\Administrator\\Desktop\\Gold\\result.json","r") as f:
    re = [json.loads(i.strip()) for i in f]
    
re.sort(key=lambda x:x["pre"],reverse=True)

In [5]:
re[:100]

[{'data': '20200215', 'static_num': 7, 'loss_num': 5, 'pre': 0.85},
 {'data': '20240825', 'static_num': 7, 'loss_num': 10, 'pre': 0.8},
 {'data': '20240215', 'static_num': 7, 'loss_num': 9, 'pre': 0.8},
 {'data': '20230215', 'static_num': 6, 'loss_num': 19, 'pre': 0.8},
 {'data': '20220215', 'static_num': 4, 'loss_num': 18, 'pre': 0.8},
 {'data': '20220215', 'static_num': 7, 'loss_num': 4, 'pre': 0.8},
 {'data': '20210215', 'static_num': 5, 'loss_num': 14, 'pre': 0.8},
 {'data': '20200825', 'static_num': 5, 'loss_num': 14, 'pre': 0.8},
 {'data': '20200825', 'static_num': 7, 'loss_num': 5, 'pre': 0.8},
 {'data': '20200215', 'static_num': 5, 'loss_num': 16, 'pre': 0.8},
 {'data': '20200215', 'static_num': 7, 'loss_num': 6, 'pre': 0.8},
 {'data': '20200215', 'static_num': 7, 'loss_num': 7, 'pre': 0.8},
 {'data': '20200215', 'static_num': 7, 'loss_num': 11, 'pre': 0.8},
 {'data': '20200215', 'static_num': 7, 'loss_num': 13, 'pre': 0.8},
 {'data': '20190825', 'static_num': 7, 'loss_num': 5,